In [1]:
from operator import truediv

import numpy as np
import code_for_hw02_downloadable as helper

In [27]:
def perceptron(data, labels, params={}, hook=None):
    # if T not in params, default to 100
    T = params.get('T', 100)

    # Your implementation here
    d, n = data.shape
    th = np.zeros((d, 1))
    th0 = np.zeros((1, 1))
    num_mistakes = 0
    for t in range(T):
        for i in range(n):
            x = data[:, i:i + 1]
            y = labels[:, i:i + 1]
            if y * (th.T @ x + th0) <= 0:
                num_mistakes +=1
                th += y * x
                th0 += y
                print(th,th0)
            if hook:
                hook((th, th0))
    print(num_mistakes)
    return th, th0

# print(perceptron(np.array([[2,3,4,5]]), np.array([[1,1,-1,-1]])))
# data = np.zeros((6, 4))
# data[1][0] = 1
# data[2][1] = 1
# data[3][2] = 1
# data[4][3] = 1
# print(data)
# labels = np.array([[1,1,-1,-1]])
data = np.identity(6)
print(data)
labels = np.array([[1,1,-1,-1,1,1]])
print(perceptron(data, labels))
# helper.test_perceptron(perceptron)



[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]
[[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]] [[1.]]
[[ 1.]
 [ 0.]
 [-1.]
 [ 0.]
 [ 0.]
 [ 0.]] [[0.]]
[[ 1.]
 [ 0.]
 [-1.]
 [-1.]
 [ 0.]
 [ 0.]] [[-1.]]
[[ 1.]
 [ 0.]
 [-1.]
 [-1.]
 [ 1.]
 [ 0.]] [[0.]]
[[ 1.]
 [ 0.]
 [-1.]
 [-1.]
 [ 1.]
 [ 1.]] [[1.]]
[[ 1.]
 [ 0.]
 [-2.]
 [-1.]
 [ 1.]
 [ 1.]] [[0.]]
[[ 1.]
 [ 1.]
 [-2.]
 [-1.]
 [ 1.]
 [ 1.]] [[1.]]
[[ 1.]
 [ 1.]
 [-2.]
 [-2.]
 [ 1.]
 [ 1.]] [[0.]]
8
(array([[ 1.],
       [ 1.],
       [-2.],
       [-2.],
       [ 1.],
       [ 1.]]), array([[0.]]))


In [3]:
def averaged_perceptron(data, labels, params={}, hook=None):
    # if T not in params, default to 100
    T = params.get('T', 100)

    # Your implementation here
    d, n = data.shape

    th = np.zeros((d, 1))
    th0 = np.zeros((1, 1))

    ths = np.zeros((d,1))
    th0s = np.zeros((1,1))
    # ths = np.zeros((d, 0))
    # th0s = np.zeros((1, 0))

    for t in range(T):
        for i in range(n):
            x = data[:, i:i + 1]
            y = labels[:, i:i + 1]
            if y * (th.T @ x + th0) <= 0:
                th += y * x
                th0 += y

            ths += th
            th0s += th0
            # ths = np.concatenate((ths, th), axis=1)
            # th0s = np.concatenate((th0s, th0), axis=1)
            if hook:
                hook((th, th0))
    return ths/(n*T), th0s/(n*T)
    # return np.mean(ths, axis=1, keepdims=True), np.mean(th0s, axis=1, keepdims=True)

helper.test_averaged_perceptron(averaged_perceptron)


-----------Test Averaged Perceptron 0-----------
Passed! 

-----------Test Averaged Perceptron 1-----------
Passed! 



In [4]:
def eval_classifier(learner, data_train, labels_train, data_test, labels_test):
    th, th0 = learner(data_train, labels_train)
    score = np.mean(np.sign(th.T @ data_test + th0.T) == labels_test, axis=1, keepdims=True)
    return score.item()

helper.test_eval_classifier(eval_classifier, perceptron)

-----------Test Eval Classifier 0-----------
Passed! 

-----------Test Eval Classifier 1-----------
Passed! 



In [5]:

def eval_learning_alg(learner, data_gen, n_train, n_test, it):
    total_score = 0
    for i in range(it):
        data_train, labels_train = data_gen(n_train)
        data_test, labels_test = data_gen(n_test)

        total_score += eval_classifier(learner, data_train, labels_train, data_test, labels_test)
    return total_score/it

helper.test_eval_learning_alg(eval_learning_alg,perceptron)

-----------Test Eval Learning Algo-----------
Passed! 



In [6]:
from code_for_hw02_downloadable import score

def xval_learning_alg(learner, data, labels, k):
    #cross validation of learning algorithm
    score_sum = 0
    data_list = np.array_split(data, k, axis=1) # data   dxn
    labels_list = np.array_split(labels, k, axis=1)   # labels 1xn
    for i in range(k):
        data_train = np.concatenate([part for j,part in enumerate(data_list) if j !=i], axis=1)
        labels_train = np.concatenate([part for j,part in enumerate(labels_list) if j !=i], axis=1)
        data_test = data_list[i]
        labels_test = labels_list[i]
        score_sum += eval_classifier(learner, data_train, labels_train, data_test, labels_test)
        # th, th0 = learner(data_train, labels_train)
        # score_sum += score(data_test, labels_test, th, th0) / data_test.shape[1]
    return score_sum / k

helper.test_xval_learning_alg(xval_learning_alg, perceptron)

-----------Test Cross-eval Learning Algo-----------
Passed! 



In [7]:
for pflip in [0.1, 0.25]:
    for alg in [perceptron, averaged_perceptron]:
        accuracy = eval_learning_alg(alg, helper.gen_flipped_lin_separable(pflip=pflip), 20, 20,100)
        print(pflip, alg.__name__, accuracy)

0.1 perceptron 0.7540000000000002
0.1 averaged_perceptron 0.8095000000000002
0.25 perceptron 0.5850000000000001
0.25 averaged_perceptron 0.6400000000000001


In [8]:
def training_accuracy(learner, data_gen, n_train, it):
    total_score = 0
    for i in range(it):
        data_train, labels_train = data_gen(n_train)
        total_score += eval_classifier(learner, data_train, labels_train, data_train, labels_train)
    return total_score/it

for pflip in [0.1, 0.25]:
    for alg in [perceptron, averaged_perceptron]:
        accuracy = training_accuracy(alg, helper.gen_flipped_lin_separable(pflip=pflip), 20,100)
        print(pflip, alg.__name__, accuracy)



0.1 perceptron 0.8099999999999997
0.1 averaged_perceptron 0.8660000000000003
0.25 perceptron 0.6679999999999998
0.25 averaged_perceptron 0.7150000000000002
